In [1]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

from deap import base, creator, tools, algorithms
import random

In [2]:
df_distance_km = "../dataset/df_distance_km.xlsx"
df = pd.read_excel(df_distance_km)
df.head()

,CLIENTES,Cliente_1,Cliente_2,Cliente_3,Cliente_4,Cliente_5,Cliente_6,Cliente_7,Cliente_8,Cliente_9,...,Cliente_12,Cliente_13,Cliente_14,Cliente_15,Cliente_16,Cliente_17,Cliente_18,Cliente_19,Cliente_20,Almacén
0,Cliente_1,0.0000,7.5625,15.5365,1.1998,4.7145,1.7407,7.9408,17.1947,4.2933,...,6.0225,5.4470,2.2133,11.1505,1.5775,10.8288,9.1456,20.4871,22.1445,3.6114
1,Cliente_2,7.5625,0.0000,3.3838,7.7433,14.5720,8.5237,0.4847,13.7974,10.1522,...,10.1049,2.6961,13.4907,18.0835,7.0275,19.8218,8.2737,9.6369,19.1038,10.7361
2,Cliente_3,15.5365,3.3838,0.0000,12.5438,0.0000,0.0000,0.0000,16.0355,13.9120,...,12.3430,5.0114,15.7289,17.9217,9.6824,22.0599,10.5118,7.7574,16.5997,13.9021
3,Cliente_4,1.1998,7.7433,12.5438,0.0000,5.0721,0.9119,7.5798,17.4095,3.5781,...,6.2330,4.7117,2.8799,11.3610,1.3127,11.1926,9.3561,20.7019,21.1518,3.3673
4,Cliente_5,4.7145,14.5720,0.0000,5.0721,0.0000,4.8187,0.0000,0.0000,0.0000,...,6.8738,9.1064,3.6476,12.0019,5.4347,5.2872,9.9969,21.3384,25.5947,4.5417


In [3]:
df_clientes = df["CLIENTES"]
df_clientes

0      Cliente_1
1      Cliente_2
2      Cliente_3
3      Cliente_4
4      Cliente_5
5      Cliente_6
6      Cliente_7
7      Cliente_8
8      Cliente_9
9     Cliente_10
10    Cliente_11
11    Cliente_12
12    Cliente_13
13    Cliente_14
14    Cliente_15
15    Cliente_16
16    Cliente_17
17    Cliente_18
18    Cliente_19
19    Cliente_20
20       Almacén
Name: CLIENTES, dtype: object

In [4]:
df = df.drop(columns=["CLIENTES"])
distance_km_array = df.to_numpy()
distance_km_array

array([[ 0.    ,  7.5625, 15.5365,  1.1998,  4.7145,  1.7407,  7.9408,
        17.1947,  4.2933,  3.2659,  2.1866,  6.0225,  5.447 ,  2.2133,
        11.1505,  1.5775, 10.8288,  9.1456, 20.4871, 22.1445,  3.6114],
       [ 7.5625,  0.    ,  3.3838,  7.7433, 14.572 ,  8.5237,  0.4847,
        13.7974, 10.1522,  7.1521, 14.8113, 10.1049,  2.6961, 13.4907,
        18.0835,  7.0275, 19.8218,  8.2737,  9.6369, 19.1038, 10.7361],
       [15.5365,  3.3838,  0.    , 12.5438,  0.    ,  0.    ,  0.    ,
        16.0355, 13.912 , 13.0649, 17.0494, 12.343 ,  5.0114, 15.7289,
        17.9217,  9.6824, 22.0599, 10.5118,  7.7574, 16.5997, 13.9021],
       [ 1.1998,  7.7433, 12.5438,  0.    ,  5.0721,  0.9119,  7.5798,
        17.4095,  3.5781,  3.3451,  2.8532,  6.233 ,  4.7117,  2.8799,
        11.361 ,  1.3127, 11.1926,  9.3561, 20.7019, 21.1518,  3.3673],
       [ 4.7145, 14.572 ,  0.    ,  5.0721,  0.    ,  4.8187,  0.    ,
         0.    ,  0.    ,  7.217 ,  2.6253,  6.8738,  9.1064,  3.6476,
  

In [5]:
df_vehiculos = pd.read_excel("../dataset/df_vehicle.xlsx")
df_vehiculos

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,1,2026,0.20,603
1,2,4362,0.14,630
2,3,4881,0.20,664
3,4,3321,0.19,514
4,5,10000,0.32,350
5,6,3129,0.14,791


In [6]:
df_time = pd.read_excel("../dataset/df_distance_min.xlsx")

#df_time= df_time.drop(columns=["CLIENTES"])
df_time_array = df.to_numpy()
df_time_array

array([[ 0.    ,  7.5625, 15.5365,  1.1998,  4.7145,  1.7407,  7.9408,
        17.1947,  4.2933,  3.2659,  2.1866,  6.0225,  5.447 ,  2.2133,
        11.1505,  1.5775, 10.8288,  9.1456, 20.4871, 22.1445,  3.6114],
       [ 7.5625,  0.    ,  3.3838,  7.7433, 14.572 ,  8.5237,  0.4847,
        13.7974, 10.1522,  7.1521, 14.8113, 10.1049,  2.6961, 13.4907,
        18.0835,  7.0275, 19.8218,  8.2737,  9.6369, 19.1038, 10.7361],
       [15.5365,  3.3838,  0.    , 12.5438,  0.    ,  0.    ,  0.    ,
        16.0355, 13.912 , 13.0649, 17.0494, 12.343 ,  5.0114, 15.7289,
        17.9217,  9.6824, 22.0599, 10.5118,  7.7574, 16.5997, 13.9021],
       [ 1.1998,  7.7433, 12.5438,  0.    ,  5.0721,  0.9119,  7.5798,
        17.4095,  3.5781,  3.3451,  2.8532,  6.233 ,  4.7117,  2.8799,
        11.361 ,  1.3127, 11.1926,  9.3561, 20.7019, 21.1518,  3.3673],
       [ 4.7145, 14.572 ,  0.    ,  5.0721,  0.    ,  4.8187,  0.    ,
         0.    ,  0.    ,  7.217 ,  2.6253,  6.8738,  9.1064,  3.6476,
  

In [7]:
df_pedidos = pd.read_excel("../dataset/df_orders.xlsx")
df_pedidos

,cliente,mes_anio,order_demand
0,Cliente_1,12-2024,909
1,Cliente_2,12-2024,959
2,Cliente_3,12-2024,960
3,Cliente_4,12-2024,980
4,Cliente_5,12-2024,979
5,Cliente_6,12-2024,908
6,Cliente_7,12-2024,924
7,Cliente_8,12-2024,920
8,Cliente_9,12-2024,886
9,Cliente_10,12-2024,964


In [8]:

def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente (almacén)
    random.shuffle(puntos)  # Mezcla los puntos
    rutas = [[] for _ in range(len(df_vehiculos))]  # Una ruta por vehículo
    pedidos_asignados = {i: 0 for i in range(len(df_vehiculos))}  # Capacidad usada por cada vehículo

    # Garantizar que cada vehículo tenga al menos un cliente
    for i in range(len(df_vehiculos)):
        if puntos:
            punto = puntos.pop()  # Asigna un punto a este vehículo
            pedido_size = df_pedidos.loc[punto, 'order_demand']
            if pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size

    # Asignar los puntos restantes
    for punto in puntos:
        pedido_size = df_pedidos.loc[punto, 'order_demand']  # Tamaño del pedido
        for i, capacidad_usada in pedidos_asignados.items():
            if capacidad_usada + pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size
                break

    # Agregar inicio y fin (almacén) a cada ruta
    for ruta in rutas:
        ruta.insert(0, ultimo_cliente)  # Inicio (almacén)
        ruta.append(ultimo_cliente)  # Fin (almacén)

    return rutas

# Función para evaluar un individuo
def evaluate(individual):
    total_distance = 0
    total_cost = 0
    penalty = 0

    for i, ruta in enumerate(individual):
        ruta_distance = 0
        carga_total = 0
        
        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]

            try:
                distance = distance_km_array[city_a][city_b]
                pedido_size = df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
            except IndexError as e:
                penalty += 1000
                continue

            if distance == 0:
                penalty += 1000
                continue

            ruta_distance += distance
            carga_total += pedido_size

        if ruta_distance > df_vehiculos.loc[i, 'autonomia_km']:
            penalty += (ruta_distance - df_vehiculos.loc[i, 'autonomia_km']) * 10
        if carga_total > df_vehiculos.loc[i, 'capacidad_kg']:
            penalty += (carga_total - df_vehiculos.loc[i, 'capacidad_kg']) * 10

        # Calcular el costo de la ruta (suponiendo un costo por kilómetro y un costo fijo por ruta)
        costo_por_km = df_vehiculos.loc[i, 'costo_por_km'] if 'costo_por_km' in df_vehiculos.columns else 1.0  # Costo por km, asumiendo 1.0 si no está definido
        costo_fijo = df_vehiculos.loc[i, 'costo_fijo'] if 'costo_fijo' in df_vehiculos.columns else 10.0  # Costo fijo por ruta, asumiendo 10.0 si no está definido
        ruta_cost = (ruta_distance * costo_por_km) + costo_fijo

        total_distance += ruta_distance
        total_cost += ruta_cost

    # Asegúrate de retornar una tupla de dos valores
    return (total_distance + penalty, total_cost)

def crossover_individuals(ind1, ind2):
    ruta1 = ind1[1:-1]  # Intermedios de ind1
    ruta2 = ind2[1:-1]  # Intermedios de ind2

    size = len(ruta1)
    p1, p2 = sorted(random.sample(range(size), 2))  # Dos puntos de cruce

    child1, child2 = ruta1[:], ruta2[:]

    # Intercambiar segmentos
    child1[p1:p2], child2[p1:p2] = ruta2[p1:p2], ruta1[p1:p2]

    # Recombinar las rutas
    return [ind1[0]] + child1 + [ind1[-1]], [ind2[0]] + child2 + [ind2[-1]]

def mutate_individual(individual):
    ruta_intermedia = individual[1:-1]  # Excluye el primer y último punto
    random.shuffle(ruta_intermedia)  # Mezcla los puntos intermedios
    return [individual[0]] + ruta_intermedia + [individual[-1]]  # Recompone la ruta


In [9]:

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # Minimizar distancia y costo
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover_individuals)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

ultimo_cliente = len(distance_km_array) - 1
population = toolbox.population(n=100)
    
ngen = 200  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruce
mutpb = 0.2  # Probabilidad de mutación

from tqdm import tqdm

for gen in tqdm(range(ngen)):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    
    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    population[:] = offspring
    
    fits = [ind.fitness.values for ind in population]
    
    min_dist = min(f[0] for f in fits)
    min_cost = min(f[1] for f in fits)
    #print(f"Generación {gen + 1}: Distancia Min = {min_dist}, Costo Min = {min_cost}")

best_ind = tools.selBest(population, 1)[0]
print("\nMejor solución encontrada:")
lista_rutas = []
total_cost = 0
for i, ruta in enumerate(best_ind):
    ruta_distance = 0
    carga_total = 0
    
    for j in range(len(ruta) - 1):
        city_a = ruta[j]
        city_b = ruta[j + 1]
        ruta_distance += distance_km_array[city_a][city_b]
        carga_total += df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
    
    costo_por_km = df_vehiculos.loc[i, 'costo_km'] if 'costo_km' in df_vehiculos.columns else 1.0
    ruta_cost = (ruta_distance * costo_por_km)
    total_cost += ruta_cost

    print(f"Vehículo {i + 1}:")
    print(f"  Ruta: {ruta}")
    print(f"  Distancia total: {ruta_distance:.2f} km (Máxima permitida: {df_vehiculos.loc[i, 'autonomia_km']} km)")
    print(f"  Carga total: {carga_total:.2f} (Máxima permitida: {df_vehiculos.loc[i, 'capacidad_kg']})")
    print(f"  Costo de la ruta: {ruta_cost:.2f}")
    diccionario = {"vehiculo": i, "ruta": ruta, "costo": ruta_cost}
    lista_rutas.append(diccionario)

print(f"\nCosto total: {total_cost:.2f}")

100%|██████████| 200/200 [00:03<00:00, 62.26it/s]


Mejor solución encontrada:
Vehículo 1:
  Ruta: [20, 7, 17, 20]
  Distancia total: 43.16 km (Máxima permitida: 603 km)
  Carga total: 1892.00 (Máxima permitida: 2026)
  Costo de la ruta: 8.63
Vehículo 2:
  Ruta: [20, 4, 14, 10, 16, 20]
  Distancia total: 44.48 km (Máxima permitida: 630 km)
  Carga total: 3827.00 (Máxima permitida: 4362)
  Costo de la ruta: 6.23
Vehículo 3:
  Ruta: [20, 5, 3, 0, 12, 15, 20]
  Distancia total: 18.69 km (Máxima permitida: 664 km)
  Carga total: 4638.00 (Máxima permitida: 4881)
  Costo de la ruta: 3.74
Vehículo 4:
  Ruta: [20, 19, 6, 9, 20]
  Distancia total: 45.30 km (Máxima permitida: 514 km)
  Carga total: 2769.00 (Máxima permitida: 3321)
  Costo de la ruta: 8.61
Vehículo 5:
  Ruta: [20, 13, 8, 2, 1, 11, 20]
  Distancia total: 46.22 km (Máxima permitida: 350 km)
  Carga total: 4711.00 (Máxima permitida: 10000)
  Costo de la ruta: 14.79
Vehículo 6:
  Ruta: [20, 18, 20]
  Distancia total: 44.07 km (Máxima permitida: 791 km)
  Carga total: 901.00 (Máxima p

In [10]:
df_vehiculos_mitad_flota = df_vehiculos.iloc[:len(df_vehiculos)//2].reset_index(drop=True)
df_vehiculos_mitad_flota

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,1,2026,0.20,603
1,2,4362,0.14,630
2,3,4881,0.20,664


## SE REDUCE LA CANTIDAD DE LA FLOTA A LA MITAD

In [11]:

def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente (almacén)
    random.shuffle(puntos)  # Mezcla los puntos
    rutas = [[] for _ in range(len(df_vehiculos_mitad_flota))]  # Una ruta por vehículo
    pedidos_asignados = {i: 0 for i in range(len(df_vehiculos_mitad_flota))}  # Capacidad usada por cada vehículo

    # Garantizar que cada vehículo tenga al menos un cliente
    for i in range(len(df_vehiculos_mitad_flota)):
        if puntos:
            punto = puntos.pop()  # Asigna un punto a este vehículo
            pedido_size = df_pedidos.loc[punto, 'order_demand']
            if pedido_size <= df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size

    # Asignar los puntos restantes
    for punto in puntos:
        pedido_size = df_pedidos.loc[punto, 'order_demand']  # Tamaño del pedido
        for i, capacidad_usada in pedidos_asignados.items():
            if capacidad_usada + pedido_size <= df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size
                break

    # Agregar inicio y fin (almacén) a cada ruta
    for ruta in rutas:
        ruta.insert(0, ultimo_cliente)  # Inicio (almacén)
        ruta.append(ultimo_cliente)  # Fin (almacén)

    return rutas

# Función para evaluar un individuo
def evaluate(individual):
    total_distance = 0
    total_cost = 0
    penalty = 0

    for i, ruta in enumerate(individual):
        ruta_distance = 0
        carga_total = 0
        
        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]

            try:
                distance = distance_km_array[city_a][city_b]
                pedido_size = df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
            except IndexError as e:
                penalty += 1000
                continue

            if distance == 0:
                penalty += 1000
                continue

            ruta_distance += distance
            carga_total += pedido_size

        if ruta_distance > df_vehiculos_mitad_flota.loc[i, 'autonomia_km']:
            penalty += (ruta_distance - df_vehiculos_mitad_flota.loc[i, 'autonomia_km']) * 10
        if carga_total > df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']:
            penalty += (carga_total - df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']) * 10

        # Calcular el costo de la ruta (suponiendo un costo por kilómetro y un costo fijo por ruta)
        costo_por_km = df_vehiculos_mitad_flota.loc[i, 'costo_por_km'] if 'costo_por_km' in df_vehiculos_mitad_flota.columns else 1.0  # Costo por km, asumiendo 1.0 si no está definido
        costo_fijo = df_vehiculos_mitad_flota.loc[i, 'costo_fijo'] if 'costo_fijo' in df_vehiculos_mitad_flota.columns else 10.0  # Costo fijo por ruta, asumiendo 10.0 si no está definido
        ruta_cost = (ruta_distance * costo_por_km) + costo_fijo

        total_distance += ruta_distance
        total_cost += ruta_cost

    # Asegúrate de retornar una tupla de dos valores
    return (total_distance + penalty, total_cost)


def crossover_individuals(ind1, ind2):
    ruta1 = ind1[1:-1]  # Intermedios de ind1
    ruta2 = ind2[1:-1]  # Intermedios de ind2

    size = len(ruta1)
    if size > 1:  # Solo si hay más de un punto para cruzar
        p1, p2 = sorted(random.sample(range(size), 2))  # Dos puntos de cruce
        child1, child2 = ruta1[:], ruta2[:]

        # Intercambiar segmentos
        child1[p1:p2], child2[p1:p2] = ruta2[p1:p2], ruta1[p1:p2]
    else:
        # Si hay 1 o menos puntos, no se puede hacer un cruce, así que clonamos las rutas sin cambios
        child1, child2 = ruta1[:], ruta2[:]

    # Recombinar las rutas
    return [ind1[0]] + child1 + [ind1[-1]], [ind2[0]] + child2 + [ind2[-1]]

def mutate_individual(individual):
    ruta_intermedia = individual[1:-1]  # Excluye el primer y último punto
    random.shuffle(ruta_intermedia)  # Mezcla los puntos intermedios
    return [individual[0]] + ruta_intermedia + [individual[-1]]  # Recompone la ruta


creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # Minimizar distancia y costo
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover_individuals)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

ultimo_cliente = len(distance_km_array) - 1
population = toolbox.population(n=100)
    
ngen = 200  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruce
mutpb = 0.2  # Probabilidad de mutación

from tqdm import tqdm

for gen in tqdm(range(ngen)):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    
    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    population[:] = offspring
    
    fits = [ind.fitness.values for ind in population]
    
    min_dist = min(f[0] for f in fits)
    min_cost = min(f[1] for f in fits)
    #print(f"Generación {gen + 1}: Distancia Min = {min_dist}, Costo Min = {min_cost}")

best_ind = tools.selBest(population, 1)[0]
print("\nMejor solución encontrada:")
lista_rutas = []
total_cost = 0
for i, ruta in enumerate(best_ind):
    ruta_distance = 0
    carga_total = 0
    
    for j in range(len(ruta) - 1):
        city_a = ruta[j]
        city_b = ruta[j + 1]
        ruta_distance += distance_km_array[city_a][city_b]
        carga_total += df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
    
    costo_por_km = df_vehiculos_mitad_flota.loc[i, 'costo_km'] if 'costo_km' in df_vehiculos_mitad_flota.columns else 1.0
    ruta_cost = (ruta_distance * costo_por_km)
    total_cost += ruta_cost

    print(f"Vehículo {i + 1}:")
    print(f"  Ruta: {ruta}")
    print(f"  Distancia total: {ruta_distance:.2f} km (Máxima permitida: {df_vehiculos_mitad_flota.loc[i, 'autonomia_km']} km)")
    print(f"  Carga total: {carga_total:.2f} (Máxima permitida: {df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']})")
    print(f"  Costo de la ruta: {ruta_cost:.2f}")
    diccionario = {"vehiculo": i, "ruta": ruta, "costo": ruta_cost}
    lista_rutas.append(diccionario)

print(f"\nCosto total: {total_cost:.2f}")

c:\Users\Manu\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Manu\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
100%|██████████| 200/200 [00:01<00:00, 112.76it/s]


Mejor solución encontrada:
Vehículo 1:
  Ruta: [20, 2, 14, 20]
  Distancia total: 47.43 km (Máxima permitida: 603 km)
  Carga total: 1898.00 (Máxima permitida: 2026)
  Costo de la ruta: 9.49
Vehículo 2:
  Ruta: [20, 13, 4, 3, 10, 20]
  Distancia total: 18.52 km (Máxima permitida: 630 km)
  Carga total: 3833.00 (Máxima permitida: 4362)
  Costo de la ruta: 2.59
Vehículo 3:
  Ruta: [20, 1, 6, 15, 5, 12, 20]
  Distancia total: 33.51 km (Máxima permitida: 664 km)
  Carga total: 4632.00 (Máxima permitida: 4881)
  Costo de la ruta: 6.70

Costo total: 18.78


FALTAN CLIENTES POR REPARTIR, PUESTO QUE CON LA MITAD DE LA FLOTA NO SE PUEDE REALIZAR TODOS LOS PEDIDOS.

## SE PRUEBA CON LA MITAD DE LA FLOTA PERO SOLO CON LOS VEHICULOS CON MAYOR CAPACIDAD

In [12]:
df_vehiculos_mitad_flota = df_vehiculos_mitad_flota.nlargest(3, 'capacidad_kg').reset_index(drop=True)
df_vehiculos_mitad_flota

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,3,4881,0.20,664
1,2,4362,0.14,630
2,1,2026,0.20,603


In [13]:

creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))  # Minimizar distancia y costo
creator.create("Individual", list, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover_individuals)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

ultimo_cliente = len(distance_km_array) - 1
population = toolbox.population(n=100)
    
ngen = 200  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruce
mutpb = 0.2  # Probabilidad de mutación

from tqdm import tqdm

for gen in tqdm(range(ngen)):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))
    
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values
    
    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values
    
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
    
    population[:] = offspring
    
    fits = [ind.fitness.values for ind in population]
    
    min_dist = min(f[0] for f in fits)
    min_cost = min(f[1] for f in fits)
    #print(f"Generación {gen + 1}: Distancia Min = {min_dist}, Costo Min = {min_cost}")

best_ind = tools.selBest(population, 1)[0]
print("\nMejor solución encontrada:")
lista_rutas = []
total_cost = 0
for i, ruta in enumerate(best_ind):
    ruta_distance = 0
    carga_total = 0
    
    for j in range(len(ruta) - 1):
        city_a = ruta[j]
        city_b = ruta[j + 1]
        ruta_distance += distance_km_array[city_a][city_b]
        carga_total += df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
    
    costo_por_km = df_vehiculos_mitad_flota.loc[i, 'costo_km'] if 'costo_km' in df_vehiculos_mitad_flota.columns else 1.0
    ruta_cost = (ruta_distance * costo_por_km)
    total_cost += ruta_cost

    print(f"Vehículo {i + 1}:")
    print(f"  Ruta: {ruta}")
    print(f"  Distancia total: {ruta_distance:.2f} km (Máxima permitida: {df_vehiculos_mitad_flota.loc[i, 'autonomia_km']} km)")
    print(f"  Carga total: {carga_total:.2f} (Máxima permitida: {df_vehiculos_mitad_flota.loc[i, 'capacidad_kg']})")
    print(f"  Costo de la ruta: {ruta_cost:.2f}")
    diccionario = {"vehiculo": i, "ruta": ruta, "costo": ruta_cost}
    lista_rutas.append(diccionario)

print(f"\nCosto total: {total_cost:.2f}")

c:\Users\Manu\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMulti' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Manu\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
100%|██████████| 200/200 [00:01<00:00, 117.97it/s]


Mejor solución encontrada:
Vehículo 1:
  Ruta: [20, 16, 6, 9, 11, 15, 20]
  Distancia total: 46.56 km (Máxima permitida: 664 km)
  Carga total: 4721.00 (Máxima permitida: 4881)
  Costo de la ruta: 9.31
Vehículo 2:
  Ruta: [20, 10, 8, 5, 0, 20]
  Distancia total: 14.70 km (Máxima permitida: 630 km)
  Carga total: 3645.00 (Máxima permitida: 4362)
  Costo de la ruta: 2.06
Vehículo 3:
  Ruta: [20, 1, 3, 20]
  Distancia total: 21.85 km (Máxima permitida: 603 km)
  Carga total: 1939.00 (Máxima permitida: 2026)
  Costo de la ruta: 4.37

Costo total: 15.74


FALTAN RUTAS POR REPARTIR PUESTO QUE NO ES POSIBLE REALIZAR TODOS LOS ENVIOS CON LA MITAD DE LA FLOTA

## caso 4

In [14]:
def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente (almacén)
    random.shuffle(puntos)  # Mezcla los puntos
    rutas = [[] for _ in range(len(df_vehiculos))]  # Una ruta por vehículo
    pedidos_asignados = {i: 0 for i in range(len(df_vehiculos))}  # Capacidad usada por cada vehículo

    # Garantizar que cada vehículo tenga al menos un cliente
    for i in range(len(df_vehiculos)):
        if puntos:
            punto = puntos.pop()  # Asigna un punto a este vehículo
            pedido_size = df_pedidos.loc[punto, 'order_demand']
            if pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size

    # Asignar los puntos restantes
    for punto in puntos:
        pedido_size = df_pedidos.loc[punto, 'order_demand']  # Tamaño del pedido
        for i, capacidad_usada in pedidos_asignados.items():
            if capacidad_usada + pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size
                break

    # Agregar inicio y fin (almacén) a cada ruta
    for ruta in rutas:
        ruta.insert(0, ultimo_cliente)  # Inicio (almacén)
        ruta.append(ultimo_cliente)  # Fin (almacén)

    return rutas

# Función para evaluar un individuo
def evaluate(individual):
    total_distance = 0
    total_time = 0
    total_cost = 0
    penalty = 0

    for i, ruta in enumerate(individual):
        ruta_distance = 0
        ruta_time = 0
        carga_total = 0
        vehicle_cost_per_km = df_vehiculos.loc[i, 'costo_km']  # Costo por kilómetro del vehículo

        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]

            try:
                distance = distance_km_array[city_a][city_b]
                time = df_time_array[city_a][city_b]
                pedido_size = df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
            except IndexError as e:
                penalty += 1000
                continue

            if distance == 0:
                penalty += 1000
                continue

            ruta_distance += distance
            ruta_time += time
            carga_total += pedido_size

        # Calcular costo de la ruta
        ruta_cost = ruta_distance * vehicle_cost_per_km
        total_cost += ruta_cost

        if ruta_distance > df_vehiculos.loc[i, 'autonomia_km']:
            penalty += (ruta_distance - df_vehiculos.loc[i, 'autonomia_km']) * 10
        if carga_total > df_vehiculos.loc[i, 'capacidad_kg']:
            penalty += (carga_total - df_vehiculos.loc[i, 'capacidad_kg']) * 10

        total_distance += ruta_distance
        total_time += ruta_time

    # Asegúrate de retornar una tupla de tres valores
    return (total_distance + penalty, total_time, total_cost)

def crossover_individuals(ind1, ind2):
    ruta1 = ind1[1:-1]  # Intermedios de ind1
    ruta2 = ind2[1:-1]  # Intermedios de ind2

    size = len(ruta1)
    p1, p2 = sorted(random.sample(range(size), 2))  # Dos puntos de cruce

    child1, child2 = ruta1[:], ruta2[:]

    # Intercambiar segmentos
    child1[p1:p2], child2[p1:p2] = ruta2[p1:p2], ruta1[p1:p2]

    # Recombinar las rutas
    return [ind1[0]] + child1 + [ind1[-1]], [ind2[0]] + child2 + [ind2[-1]]

def mutate_individual(individual):
    ruta_intermedia = individual[1:-1]  # Excluye el primer y último punto
    random.shuffle(ruta_intermedia)  # Mezcla los puntos intermedios
    return [individual[0]] + ruta_intermedia + [individual[-1]],  # Recompone la ruta


In [15]:

##CASO 4 CON VARIABLE TIEMPO
# Crear tipos personalizados en DEAP
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0, -1.0))  # Minimizar distancia, tiempo y costo
creator.create("Individual", list, fitness=creator.FitnessMulti)

# Definición de la función para generar individuos
def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente (almacén)
    random.shuffle(puntos)  # Mezcla los puntos
    rutas = [[] for _ in range(len(df_vehiculos))]  # Una ruta por vehículo
    pedidos_asignados = {i: 0 for i in range(len(df_vehiculos))}  # Capacidad usada por cada vehículo

    # Garantizar que cada vehículo tenga al menos un cliente
    for i in range(len(df_vehiculos)):
        if puntos:
            punto = puntos.pop()  # Asigna un punto a este vehículo
            pedido_size = df_pedidos.loc[punto, 'order_demand']
            if pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size

    # Asignar los puntos restantes
    for punto in puntos:
        pedido_size = df_pedidos.loc[punto, 'order_demand']  # Tamaño del pedido
        for i, capacidad_usada in pedidos_asignados.items():
            if capacidad_usada + pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size
                break

    # Agregar inicio y fin (almacén) a cada ruta
    for ruta in rutas:
        ruta.insert(0, ultimo_cliente)  # Inicio (almacén)
        ruta.append(ultimo_cliente)  # Fin (almacén)

    return rutas

# Función para evaluar un individuo
def evaluate(individual):
    total_distance = 0
    total_time = 0
    total_cost = 0
    penalty = 0

    for i, ruta in enumerate(individual):
        ruta_distance = 0
        ruta_time = 0
        ruta_cost = 0
        carga_total = 0

        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]

            try:
                distance = distance_km_array[city_a][city_b]
                time = df_time_array[city_a][city_b]
                pedido_size = df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
            except IndexError:
                penalty += 1000
                continue

            if distance == 0:
                penalty += 1000
                continue

            ruta_distance += distance
            ruta_time += time
            carga_total += pedido_size
            ruta_cost += distance * df_vehiculos.loc[i, 'costo_km']

        if ruta_distance > df_vehiculos.loc[i, 'autonomia_km']:
            penalty += (ruta_distance - df_vehiculos.loc[i, 'autonomia_km']) * 10
        if carga_total > df_vehiculos.loc[i, 'capacidad_kg']:
            penalty += (carga_total - df_vehiculos.loc[i, 'capacidad_kg']) * 10

        total_distance += ruta_distance
        total_time += ruta_time
        total_cost += ruta_cost

    return (total_distance + penalty, total_time, total_cost)

# Operadores genéticos
def crossover_individuals(ind1, ind2):
    ruta1 = ind1[1:-1]  # Intermedios de ind1
    ruta2 = ind2[1:-1]  # Intermedios de ind2

    size = len(ruta1)
    p1, p2 = sorted(random.sample(range(size), 2))  # Dos puntos de cruce

    child1, child2 = ruta1[:], ruta2[:]

    # Intercambiar segmentos
    child1[p1:p2], child2[p1:p2] = ruta2[p1:p2], ruta1[p1:p2]

    # Recombinar las rutas
    return [ind1[0]] + child1 + [ind1[-1]], [ind2[0]] + child2 + [ind2[-1]]

def mutate_individual(individual):
    ruta_intermedia = individual[1:-1]  # Excluye el primer y último punto
    random.shuffle(ruta_intermedia)  # Mezcla los puntos intermedios
    return [individual[0]] + ruta_intermedia + [individual[-1]]

# Configuración de DEAP
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", crossover_individuals)
toolbox.register("mutate", mutate_individual)
toolbox.register("select", tools.selTournament, tournsize=3)

# Algoritmo evolutivo
ultimo_cliente = len(distance_km_array) - 1
population = toolbox.population(n=200)

ngen = 200  # Número de generaciones
cxpb = 0.7  # Probabilidad de cruce
mutpb = 0.2  # Probabilidad de mutación

for gen in tqdm(range(ngen)):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < cxpb:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < mutpb:
            toolbox.mutate(mutant)
            del mutant.fitness.values

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring

    fits = [ind.fitness.values for ind in population]

    min_dist = min(f[0] for f in fits)
    min_time = min(f[1] for f in fits)
    min_cost = min(f[2] for f in fits)
    # print(f"Generación {gen + 1}: Distancia Min = {min_dist}, Tiempo Min = {min_time}, Costo Min = {min_cost}")

# Mostrar la mejor solución
best_ind = tools.selBest(population, 1)[0]
print("\nMejor solución encontrada:")
lista_rutas2 = []
total_cost = 0

for i, ruta in enumerate(best_ind):
    ruta_distance = 0
    ruta_time = 0
    ruta_cost = 0
    carga_total = 0

    for j in range(len(ruta) - 1):
        city_a = ruta[j]
        city_b = ruta[j + 1]
        ruta_distance += distance_km_array[city_a][city_b]
        ruta_time += df_time_array[city_a][city_b]
        carga_total += df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
        ruta_cost += distance_km_array[city_a][city_b] * df_vehiculos.loc[i, 'costo_km']

    total_cost += ruta_cost

    print(f"Vehículo {i + 1}:")
    print(f"  Ruta: {ruta}")
    print(f"  Distancia total: {ruta_distance:.2f} km (Máxima permitida: {df_vehiculos.loc[i, 'autonomia_km']} km)")
    print(f"  Tiempo total: {ruta_time:.2f} minutos")
    print(f"  Carga total: {carga_total:.2f} (Máxima permitida: {df_vehiculos.loc[i, 'capacidad_kg']})")
    print(f"  Costo total: {ruta_cost:.2f} (Costo por km: {df_vehiculos.loc[i, 'costo_km']})")
    diccionario = {"vehiculo" : i, "ruta" : ruta}
    lista_rutas2.append(diccionario)

print(f"\nCosto total de la solución: {total_cost:.2f}")


100%|██████████| 200/200 [00:09<00:00, 21.65it/s]


Mejor solución encontrada:
Vehículo 1:
  Ruta: [20, 10, 1, 20]
  Distancia total: 28.46 km (Máxima permitida: 603 km)
  Tiempo total: 28.46 minutos
  Carga total: 1901.00 (Máxima permitida: 2026)
  Costo total: 5.69 (Costo por km: 0.2)
Vehículo 2:
  Ruta: [20, 19, 17, 13, 4, 20]
  Distancia total: 55.99 km (Máxima permitida: 630 km)
  Tiempo total: 55.99 minutos
  Carga total: 3764.00 (Máxima permitida: 4362)
  Costo total: 7.84 (Costo por km: 0.14)
Vehículo 3:
  Ruta: [20, 16, 15, 7, 14, 11, 20]
  Distancia total: 56.35 km (Máxima permitida: 664 km)
  Tiempo total: 56.35 minutos
  Carga total: 4691.00 (Máxima permitida: 4881)
  Costo total: 11.27 (Costo por km: 0.2)
Vehículo 4:
  Ruta: [20, 5, 3, 8, 20]
  Distancia total: 8.23 km (Máxima permitida: 514 km)
  Tiempo total: 8.23 minutos
  Carga total: 2774.00 (Máxima permitida: 3321)
  Costo total: 1.56 (Costo por km: 0.19)
Vehículo 5:
  Ruta: [20, 0, 2, 18, 12, 9, 20]
  Distancia total: 50.30 km (Máxima permitida: 350 km)
  Tiempo tot

In [16]:
df_locations = pd.read_excel("../dataset/df_location.xlsx")
df_locations

,Cliente,Latitud,Longitud
0,Cliente_1,40.402828,-3.696878
1,Cliente_2,40.455820,-3.700350
2,Cliente_3,40.471840,-3.708079
3,Cliente_4,40.404688,-3.706262
4,Cliente_5,40.371990,-3.695056
5,Cliente_6,40.400481,-3.712361
6,Cliente_7,40.457059,-3.703511
7,Cliente_8,40.446872,-3.586515
8,Cliente_9,40.389637,-3.735354
9,Cliente_10,40.409083,-3.676125


In [17]:

lista_rutas


for elemento in lista_rutas:
    lista_rutas_tuplas_localizaciones = []
    for rut in elemento['ruta']:
        latitud = df_locations.loc[rut, 'Latitud']
        longitud = df_locations.loc[rut, 'Longitud']
        lista_rutas_tuplas_localizaciones.append((latitud, longitud))
    elemento['ruta'] = lista_rutas_tuplas_localizaciones
lista_rutas

df_rutas_elementos = pd.DataFrame(lista_rutas)
df_rutas_elementos

,vehiculo,ruta,costo
0,0,"[(40.3885963, -3.7270384), (40.3340239, -3.711...",9.312260
1,1,"[(40.3885963, -3.7270384), (40.3893878, -3.700...",2.058196
2,2,"[(40.3885963, -3.7270384), (40.4558202, -3.700...",4.369340


In [18]:
df_rutas_elementos

,vehiculo,ruta,costo
0,0,"[(40.3885963, -3.7270384), (40.3340239, -3.711...",9.312260
1,1,"[(40.3885963, -3.7270384), (40.3893878, -3.700...",2.058196
2,2,"[(40.3885963, -3.7270384), (40.4558202, -3.700...",4.369340


## MAPA DE LAS RUTAS SIN TENER EN CUENTA EL TIEMPO

In [19]:

import folium

# Crear el mapa base centrado en las coordenadas medias de las rutas
coordenadas = [coord for ruta in df_rutas_elementos['ruta'] for coord in ruta]
latitudes = [lat for lat, lon in coordenadas]
longitudes = [lon for lat, lon in coordenadas]

m = folium.Map(location=[sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)], zoom_start=14)

# Función para calcular un punto intermedio
def punto_intermedio(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    return (lat1 + lat2) / 2, (lon1 + lon2) / 2

# Crear capas para cada vehículo
for _, row in df_rutas_elementos.iterrows():
    vehiculo = row['vehiculo']
    ruta = row['ruta']
    
    # Crear una FeatureGroup para cada vehículo
    feature_group = folium.FeatureGroup(name=f'Ruta {vehiculo}')
    
    # Añadir la línea que conecta los puntos de la ruta
    folium.PolyLine(ruta, color='blue', weight=2.5, opacity=1).add_to(feature_group)
    
    # Añadir flechas en los segmentos de la ruta
    for i in range(len(ruta) - 1):
        punto_inicio = ruta[i]
        punto_fin = ruta[i + 1]
        intermedio = punto_intermedio(punto_inicio, punto_fin)
        
        # Añadir una flecha en el punto intermedio
        folium.Marker(
            location=intermedio,
            icon=folium.DivIcon(html="""<div style="transform: rotate(-45deg); font-size: 16px;">→</div>"""),
            tooltip=f'Flecha {vehiculo}'
        ).add_to(feature_group)
    df_locations
    # Añadir marcadores en los puntos de la ruta con el cliente y el orden de visita
    for idx, (lat, lon) in enumerate(ruta, start=1):
        # Buscar el cliente correspondiente en df_clientes
        cliente = df_locations[
            (df_locations['Latitud'] == lat) & 
            (df_locations['Longitud'] == lon)
        ]['Cliente'].values[0]
        
        folium.Marker(
            location=[lat, lon],
            tooltip=f'{vehiculo} - {cliente} - Orden: {idx}',  # Orden de visita y cliente
            icon=folium.DivIcon(html=f"""
                <div style="font-size: 12px; color: black; text-align: center;">
                    <strong>{idx}</strong>
                </div>""")  # Mostrar el número del orden
        ).add_to(feature_group)
    
    # Añadir la ruta al mapa
    feature_group.add_to(m)

# Añadir un control de capas para las pestañas
folium.LayerControl().add_to(m)

# Guardar el mapa en un archivo HTML
m.save("mapa_con_flechas_y_clientes.html")

# Mostrar el mapa (si estás en Jupyter Notebook)
m

Faltaría el cliente 10 puesto que no sería posible añadirlo a cualquiera de las rutas para poder hacerla sin tener que volver al almacén.

In [20]:

for elemento in lista_rutas2:
    lista_rutas_tuplas_localizaciones = []
    for rut in elemento['ruta']:
        latitud = df_locations.loc[rut, 'Latitud']
        longitud = df_locations.loc[rut, 'Longitud']
        lista_rutas_tuplas_localizaciones.append((latitud, longitud))
    elemento['ruta'] = lista_rutas_tuplas_localizaciones
lista_rutas2

df_rutas_elementos2 = pd.DataFrame(lista_rutas2)
df_rutas_elementos2

,vehiculo,ruta
0,0,"[(40.3885963, -3.7270384), (40.3893878, -3.700..."
1,1,"[(40.3885963, -3.7270384), (40.4678501, -3.814..."
2,2,"[(40.3885963, -3.7270384), (40.3340239, -3.711..."
3,3,"[(40.3885963, -3.7270384), (40.4004807, -3.712..."
4,4,"[(40.3885963, -3.7270384), (40.4028283, -3.696..."
5,5,"[(40.3885963, -3.7270384), (40.4570586, -3.703..."


In [21]:
df_rutas_elementos2

,vehiculo,ruta
0,0,"[(40.3885963, -3.7270384), (40.3893878, -3.700..."
1,1,"[(40.3885963, -3.7270384), (40.4678501, -3.814..."
2,2,"[(40.3885963, -3.7270384), (40.3340239, -3.711..."
3,3,"[(40.3885963, -3.7270384), (40.4004807, -3.712..."
4,4,"[(40.3885963, -3.7270384), (40.4028283, -3.696..."
5,5,"[(40.3885963, -3.7270384), (40.4570586, -3.703..."


## MAPA RUTAS CONTANDO CON LA VARIABLE TIEMPO

In [22]:
coordenadas = [coord for ruta in df_rutas_elementos2['ruta'] for coord in ruta]
latitudes = [lat for lat, lon in coordenadas]
longitudes = [lon for lat, lon in coordenadas]

m = folium.Map(location=[sum(latitudes) / len(latitudes), sum(longitudes) / len(longitudes)], zoom_start=14)

# Función para calcular un punto intermedio
def punto_intermedio(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    return (lat1 + lat2) / 2, (lon1 + lon2) / 2

# Crear capas para cada vehículo
for _, row in df_rutas_elementos2.iterrows():
    vehiculo = row['vehiculo']
    ruta = row['ruta']
    
    # Crear una FeatureGroup para cada vehículo
    feature_group = folium.FeatureGroup(name=f'Ruta {vehiculo}')
    
    # Añadir la línea que conecta los puntos de la ruta
    folium.PolyLine(ruta, color='blue', weight=2.5, opacity=1).add_to(feature_group)
    
    # Añadir flechas en los segmentos de la ruta
    for i in range(len(ruta) - 1):
        punto_inicio = ruta[i]
        punto_fin = ruta[i + 1]
        intermedio = punto_intermedio(punto_inicio, punto_fin)
        
        # Añadir una flecha en el punto intermedio
        folium.Marker(
            location=intermedio,
            icon=folium.DivIcon(html="""<div style="transform: rotate(-45deg); font-size: 16px;">→</div>"""),
            tooltip=f'Flecha {vehiculo}'
        ).add_to(feature_group)
    df_locations
    # Añadir marcadores en los puntos de la ruta con el cliente y el orden de visita
    for idx, (lat, lon) in enumerate(ruta, start=1):
        # Buscar el cliente correspondiente en df_clientes
        cliente = df_locations[
            (df_locations['Latitud'] == lat) & 
            (df_locations['Longitud'] == lon)
        ]['Cliente'].values[0]
        
        folium.Marker(
            location=[lat, lon],
            tooltip=f'{vehiculo} - {cliente} - Orden: {idx}',  # Orden de visita y cliente
            icon=folium.DivIcon(html=f"""
                <div style="font-size: 12px; color: black; text-align: center;">
                    <strong>{idx}</strong>
                </div>""")  # Mostrar el número del orden
        ).add_to(feature_group)
    
    # Añadir la ruta al mapa
    feature_group.add_to(m)

# Añadir un control de capas para las pestañas
folium.LayerControl().add_to(m)

# Guardar el mapa en un archivo HTML
m.save("mapa_con_flechas_y_clientes.html")

# Mostrar el mapa (si estás en Jupyter Notebook)
m